# Binance

- Créér un clé API : https://www.binance.com/en/support/faq/360002502072
- **Quand elle est créée garder en mémoire l'api key et la secret KEY**

- créer un fichier config.cfg dans le même répertoire que le notebook et remplacer les valeurs par celles de l'API créée

```cfg
[BINANCE]
API_KEY = YOUR KEY                  # A MODIFIER
SECRET_KEY = YOUR SECRET KEY        # A MODIFIER
```

- installer le package python-binance 

```shell
pip install python-binance
ou
py -m pip install python-binance
ou autre
```
- Editer la list my_coins avec vos crypto

```python
#exemple
my_coins = ['BTC','USDT', 'ETH']     # A ADAPTER
```

- Editer le dict stack car l'API ne sait pas encore récupérer les crypto stackées 

```python
#exemple
stack = {'DOT':3, 'ADA':6.1938, 'CAKE':3.125, 'TRX': 67}      # A ADAPTER
```

- et lancer tout les cellules
! Tout est en dollars


In [257]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import requests, json, time, hashlib,hmac, configparser

# A ADAPTER
my_coins = ['BTC','USDT', 'ETH', 'ADA', 'DOT', 'CAKE', 'MATIC', 'VET', 'BNB', 'LTC', 'KSM', 'THETA', 'AVAX', 'DENT', 'EGLD', 'HOT', 'TRX', 'ONE', 'BAND', 'SXP', 'LINK']  
my_symbols = [coin + 'USDT' for coin in my_coins]

config = configparser.ConfigParser()
config.read_file(open('config.cfg'))

KEY = config.get('BINANCE', 'API_KEY')
SECRET_KEY = config.get('BINANCE', 'SECRET_KEY')
BASE_URL = 'https://api.binance.com/'

client = Client(KEY, SECRET_KEY)


In [258]:
from urllib.parse import urlencode

# fonctions utilitaires

def get_timestamp():
    return int(time.time() * 1000)

def hashing(query_string):
    return hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update({
        'Content-Type': 'application/json;charset=utf-8',
        'X-MBX-APIKEY': KEY
    })
    return {
        'GET': session.get,
        'DELETE': session.delete,
        'PUT': session.put,
        'POST': session.post,
    }.get(http_method, 'GET')

def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = 'timestamp={}'.format(get_timestamp())

    url = BASE_URL + url_path + '?' + query_string + '&signature=' + hashing(query_string)
    print("{} {}".format(http_method, url))
    params = {'url': url, 'params': {}}
    response = dispatch_request(http_method)(**params)
    return response.json()


In [259]:
# Create wallet
def create_wallet():
    wallet = {}
    for asset in my_coins:
        wallet[asset] = {'Qty':0, 'Price':0, 'Current_market_price':0, 'Saving_value':0, 'Spot_value':0, 'Stack_value':0, 'Balance':0, 'ATH': 0, 'ATH_percent':0}
    return wallet

In [260]:
# Get order history
def update_order_history():
    for symbol in my_symbols:
        if symbol != 'USDTUSDT':
            orders_history = client.get_all_orders(symbol=symbol)
            sum_coin = 0.0
            sum_cost = 0.0
            for order in orders_history:
                sum_cost += float(order['executedQty']) * float(order['price'])
                sum_coin += float(order['executedQty']) 
            wallet[symbol.replace('USDT', '')]['Qty'] = sum_coin
            wallet[symbol.replace('USDT', '')]['Price'] =sum_cost

In [261]:
def update_market_price():
    all_coins = client.get_all_tickers()
    for asset in my_coins:
        if asset != 'USDT':
            for symbol in all_coins:
                if symbol['symbol'] == asset+'USDT':
                    wallet[asset]['Current_market_price'] = float(symbol['price'])

In [262]:
# Get current Savings
def update_saving():
    saving = send_signed_request('GET', 'sapi/v1/lending/union/account')
    savings = saving['positionAmountVos']
    for asset in savings:
        wallet[asset['asset']]["Saving_value"] = float(asset['amountInUSDT'])

In [263]:
# stop value
def update_spot():
    spot = send_signed_request('GET', 'api/v3/account')

    for asset in my_coins:
        for asset_spot in spot["balances"]:
            if asset_spot['asset'] == asset:
                    wallet[asset]['Spot_value'] = wallet[asset]['Current_market_price'] * float(asset_spot['free'])

In [264]:
# get stacking value
def update_stacking():    
    # A ADAPTER
    stack = {'DOT':2.50085407+1.12174, 'ADA':6.1938, 'CAKE':3.125, 'TRX': 67}
    for asset in my_coins:
        try:
            coin = stack[asset]
            wallet[asset]['Stack_value'] = wallet[asset]['Current_market_price'] * coin
        except:
            pass

In [265]:
# balance calcul
def update_balance():
    for asset in my_coins:
        wallet[asset]["Balance"] =  wallet[asset]["Stack_value"] + wallet[asset]["Saving_value"] + wallet[asset]["Spot_value"] -  wallet[asset]["Price"]

In [266]:
# Get ATH
def update_ath():
    for asset in my_coins:
        if asset != 'USDT':
            earliest_timestamp = client._get_earliest_valid_timestamp(asset + 'USDT', '1d')  # Here "ETHUSDT" is a trading pair and "1d" is time interval
            candle = client.get_historical_klines(asset + 'USDT', Client.KLINE_INTERVAL_1WEEK, "1 May, 2021")
            max_asset = 0
            for c in candle:
                if float(c[2]) > max_asset:
                    max_asset = float(c[2])
            wallet[asset]['ATH'] = max_asset
            wallet[asset]['ATH_percent'] = wallet[asset]['Current_market_price'] / wallet[asset]['ATH'] 

In [267]:
# Long update
wallet = create_wallet()
update_market_price()
update_order_history()
update_ath()


In [268]:
# Quick update
update_market_price()
update_saving()
update_spot()
update_stacking()
update_balance()

wallet

GET https://api.binance.com/sapi/v1/lending/union/account?timestamp=1622134071259&signature=1cf61cb2e6ef7ea5e4fb491ca784c0dcac0c45682800842334feba8d6ed00122
GET https://api.binance.com/api/v3/account?timestamp=1622134071595&signature=6a2a0ceebd1705d972412dc9ec714132589d524d61560c594135ac3093641d17


{'BTC': {'Qty': 0.004071,
  'Price': 160.29399999999998,
  'Current_market_price': 38993.46,
  'Saving_value': 158.60786694,
  'Spot_value': 0.0050691498,
  'Stack_value': 0,
  'Balance': -1.6810639101999811,
  'ATH': 59500.0,
  'ATH_percent': 0.6554618487394959},
 'USDT': {'Qty': 0,
  'Price': 0,
  'Current_market_price': 0,
  'Saving_value': 107.10418394,
  'Spot_value': 0.0,
  'Stack_value': 0,
  'Balance': 107.10418394,
  'ATH': 0,
  'ATH_percent': 0},
 'ETH': {'Qty': 0.01458,
  'Price': 44.96249999999999,
  'Current_market_price': 2778.39,
  'Saving_value': 40.43025413,
  'Spot_value': 0.00016670339999999998,
  'Stack_value': 0,
  'Balance': -4.532079166599992,
  'ATH': 4372.72,
  'ATH_percent': 0.6356066704476847},
 'ADA': {'Qty': 6.2,
  'Price': 10.0626,
  'Current_market_price': 1.6962,
  'Saving_value': 0.0,
  'Spot_value': 0.007325277168,
  'Stack_value': 10.505923560000001,
  'Balance': 0.45064883716800175,
  'ATH': 2.47,
  'ATH_percent': 0.6872064777327935},
 'DOT': {'Qty':

In [269]:
## Bilan

# global balance
balance = 0
total_usdt_invest = 0
total_spot = 0
total_saving = 0
total_stacking = 0
current_value = 0

for asset in wallet:
    if asset != "USDT":
        balance += wallet[asset]['Balance']
        total_usdt_invest +=  wallet[asset]['Price']
        total_spot += wallet[asset]['Spot_value']
        total_saving += wallet[asset]['Saving_value']
        total_stacking += wallet[asset]['Stack_value']
    else:
        total_usdt_invest +=wallet[asset]['Balance']
        total_spot += wallet[asset]['Spot_value']
        total_saving += wallet[asset]['Saving_value']
        total_stacking += wallet[asset]['Stack_value']

current_value = total_spot + total_saving + total_stacking

print('total_usdt_invest',total_usdt_invest,'$')
print('current_value',current_value,'$')
print('balance',balance,'$')
print('total_spot',total_spot,'$')
print('total_saving',total_saving,'$')
print('total_stacking',total_stacking,'$')

total_usdt_invest 701.7344739399997 $
current_value 612.9634937034577 $
balance -88.77098023654217 $
total_spot 68.51257189431782 $
total_saving 387.22219622999995 $
total_stacking 157.22872557914 $


In [270]:

earliest_timestamp = client._get_earliest_valid_timestamp('ETHUSDT', '1d')  # Here "ETHUSDT" is a trading pair and "1d" is time interval

candle = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1WEEK, "1 May, 2021")
max_asset = 0
for c in candle:
    if float(c[2]) > max_asset:
        max_asset = float(c[2])
print(max_asset)

59500.0


In [271]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import IntProgress, widgets

def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

c1='#FF4600'
c2='#3DD600' 

color = colorFader(c1,c2,mix=0.5)
for asset in my_coins:
    value=wallet[asset]['Current_market_price']
    maxi=wallet[asset]['ATH']
    color = colorFader(c1,c2,mix=wallet[asset]['ATH_percent'])
    
    display(widgets.FloatProgress(
        value=value,
        min=0,
        max=maxi,
        description=asset,
        bar_style='success',
        style={'bar_color': color},
        orientation='horizontal',
    readout=True,
    readout_format='.1f',
    disabled=False
    ))
    
    display(widgets.HTML(
    value = asset + ": "+str(wallet[asset]['Current_market_price']) + ' / '+ str(wallet[asset]['ATH'])
    ))


FloatProgress(value=38993.46, bar_style='success', description='BTC', max=59500.0, style=ProgressStyle(bar_col…

HTML(value='BTC: 38993.46 / 59500.0')

FloatProgress(value=0.0, bar_style='success', description='USDT', max=0.0, style=ProgressStyle(bar_color='#ff4…

HTML(value='USDT: 0 / 0')

FloatProgress(value=2778.39, bar_style='success', description='ETH', max=4372.72, style=ProgressStyle(bar_colo…

...

HTML(value='BAND: 9.111 / 20.673')

FloatProgress(value=2.344, bar_style='success', description='SXP', max=5.87, style=ProgressStyle(bar_color='#b…

HTML(value='SXP: 2.344 / 5.87')

FloatProgress(value=32.89, bar_style='success', description='LINK', max=53.0, style=ProgressStyle(bar_color='#…

HTML(value='LINK: 32.89 / 53.0')